# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([165])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label == 2:
        x.append(example)
        y.append(-1)
    elif label == 7:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[1, 2])
    ArbitraryUnitary(params[15:30], wires=[3, 4])
    ArbitraryUnitary(params[30:45], wires=[5, 6])

    # Second layer
    ArbitraryUnitary(params[45:60], wires=[0, 1])
    ArbitraryUnitary(params[60:75], wires=[2, 3])
    ArbitraryUnitary(params[75:90], wires=[4, 5])
    ArbitraryUnitary(params[90:105], wires=[6, 7])

    # Third layer
    ArbitraryUnitary(params[105:120], wires=[2, 5])

    # Fourth layer
    ArbitraryUnitary(params[120:135], wires=[1, 2])
    ArbitraryUnitary(params[135:150], wires=[5, 6])

    # Fifth layer
    ArbitraryUnitary(params[150:165], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549
 0.84869439 0.53066622 0.49383494 0.65230764 0.22667618 0.61716246
 0.2005521  0.49093199 0.94852991 0.98197027 0.17315162 0.74251972
 0.38646803 0.65110712 0.7434636  0.24777039 0.49229597 0.42474983
 0.29006936 0.98668888 0.33945016 0.1997036

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.4946996466431095
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.7467608951707891
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.7832744405182568
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.7632508833922261
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7620730270906949
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7985865724381626
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.784452296819788
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.784452296819788
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76

epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7915194346289752
test_accuracies: [0.784452296819788, 0.784452296819788, 0.7773851590106007, 0.7974087161366313, 0.8209658421672557, 0.8303886925795053, 0.7974087161366313, 0.7915194346289752, 0.7926972909305064, 0.773851590106007, 0.7950530035335689, 0.7809187279151943, 0.7879858657243817, 0.784452296819788, 0.7785630153121319, 0.8056537102473498, 0.8138987043580683, 0.8303886925795053, 0.8080094228504122, 0.8292108362779741, 0.7891637220259129, 0.8256772673733804, 0.8209658421672557, 0.817432273262662, 0.7926972909305064, 0.8292108362779741, 0.8374558303886925, 0.8186101295641932, 0.8256772673733804, 0.7915194346289752]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.823321554770318
test_accuracies: [0.784452296819788, 0.7773851590106007, 0.7974087161366313, 0.8209658421672557, 0.8303886925795053, 0.7974087161366313, 0.7915194346289752, 0.7926972909305064, 0.77385159010

epoch 459
epoch 460
accuracy: 0.8327444051825678
test_accuracies: [0.7809187279151943, 0.7879858657243817, 0.784452296819788, 0.7785630153121319, 0.8056537102473498, 0.8138987043580683, 0.8303886925795053, 0.8080094228504122, 0.8292108362779741, 0.7891637220259129, 0.8256772673733804, 0.8209658421672557, 0.817432273262662, 0.7926972909305064, 0.8292108362779741, 0.8374558303886925, 0.8186101295641932, 0.8256772673733804, 0.7915194346289752, 0.823321554770318, 0.7997644287396938, 0.7314487632508834, 0.8221436984687869, 0.833922261484099, 0.823321554770318, 0.8303886925795053, 0.8386336866902238, 0.8244994110718492, 0.817432273262662, 0.8327444051825678]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7891637220259129
test_accuracies: [0.7879858657243817, 0.784452296819788, 0.7785630153121319, 0.8056537102473498, 0.8138987043580683, 0.8303886925795053, 0.8080094228504122, 0.8292108362779741, 0.7891637220259129, 0.82567726737

epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.8256772673733804
test_accuracies: [0.817432273262662, 0.7926972909305064, 0.8292108362779741, 0.8374558303886925, 0.8186101295641932, 0.8256772673733804, 0.7915194346289752, 0.823321554770318, 0.7997644287396938, 0.7314487632508834, 0.8221436984687869, 0.833922261484099, 0.823321554770318, 0.8303886925795053, 0.8386336866902238, 0.8244994110718492, 0.817432273262662, 0.8327444051825678, 0.7891637220259129, 0.8327444051825678, 0.8292108362779741, 0.8209658421672557, 0.7891637220259129, 0.7879858657243817, 0.8197879858657244, 0.8162544169611308, 0.8115429917550059, 0.817432273262662, 0.8032979976442873, 0.8256772673733804]
epoch 581
epoch 582
epoch 583
epoch 584
epoch 585
epoch 586
epoch 587
epoch 588
epoch 589
epoch 590
accuracy: 0.8209658421672557
test_accuracies: [0.7926972909305064, 0.8292108362779741, 0.8374558303886925, 0.8186101295641932, 0.8256772673733804, 0.7915194346

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 2.04610650e-01  1.69518454e+00  8.28431976e-01  5.33217137e-01
  1.21713532e+00  2.03845598e-01  1.68069250e-03 -1.98674108e-02
  1.68101013e-01  1.51403285e+00 -1.77563378e-01  4.74097281e-02
 -5.18789007e-02  5.30409863e-01 -1.00151630e-01  1.53272706e-01
  6.36826101e-01 -4.75625303e-01  3.88028999e-01  1.82459338e+00
  1.86467342e+00 -5.51830412e-01  8.32725824e-02  1.23747992e+00
  1.84684129e+00  9.36096826e-01  1.68943447e+00  3.38003806e-01
  1.39443846e+00  8.88240796e-01  1.31697244e+00  8.21594885e-01
  1.26399305e+00  6.68835728e-01 -5.90517217e-01  2.39452348e-01
 -6.19650187e-01  1.01927072e-01  9.56553925e-02  4.51620726e-01
  2.51036367e-01 -1.63756382e-01  2.54933478e-02  2.27283600e-01
  4.22799267e-01  5.52717360e-01  1.41699458e+00  1.32012886e+00
  1.39322644e+00  1.05359255e-01  6.53290314e-01  3.55434054e-01
  4.77462071e-01  1.58197098e+00  1.17435815e+00 -3.05887522e-02
  1.97773480e+00  1.17172717e+00  9.25961644e-01  6.69964705e-0

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.781614559029398


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 2.6397199630737305
training time: 5990.727999687195
test time: 159.53838229179382
total time: 6152.906101942062
